In [ ]:
#!pip install tensorflow-gpu

In [2]:
import tensorflow as tf
#tf.enable_eager_execution()
print(tf.__version__)

2.0.0


In [3]:
import tensorflow_datasets as tfds
imdb,info = tfds.load('imdb_reviews', with_info = True, as_supervised = True)


In [4]:
train_data, test_data = imdb['train'], imdb['test']
train_s = []
train_l = []

test_s = []
test_l = []

for s,l in train_data:
  train_s.append(str(s.numpy()))
  train_l.append(l.numpy())

for s,l in test_data:
  test_s.append(str(s.numpy()))
  test_l.append(l.numpy())


In [5]:
import numpy as np
training_l_final = np.array(train_l)
testing_l_final = np.array(test_l)



In [6]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
words = 10000
oov = '<OOV>'
padding = 'post'
length = 120
trunc = 'post'



In [7]:
tk = Tokenizer(num_words=words,oov_token = oov)
tk.fit_on_texts(train_s)
wi = tk.word_index 
train_seq = tk.texts_to_sequences(train_s)
train_pad = pad_sequences(train_seq,maxlen = length,padding = padding,truncating=trunc)
test_seq = tk.texts_to_sequences(test_s)
test_pad = pad_sequences(test_seq,maxlen = length,padding = padding,truncating=trunc)



In [8]:
# reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
# def decode_review(text):
#     return ' '.join([reverse_word_index.get(i, '?') for i in text])
# print(decode_review(padded[1]))
# print(train_sentences[1])


In [9]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(words, 16, input_length=length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()



Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 120, 16)           160000    
_________________________________________________________________
flatten (Flatten)            (None, 1920)              0         
_________________________________________________________________
dense (Dense)                (None, 6)                 11526     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [10]:
num_epochs = 10
model.fit(train_pad, training_l_final, epochs=num_epochs, validation_data=(test_pad, testing_l_final))



Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 12s 465us/sample - loss: 0.4845 - accuracy: 0.7454 - val_loss: 0.4098 - val_accuracy: 0.8099
Epoch 2/10
25000/25000 [==============================] - 8s 328us/sample - loss: 0.2381 - accuracy: 0.9089 - val_loss: 0.4129 - val_accuracy: 0.8196
Epoch 3/10
25000/25000 [==============================] - 7s 296us/sample - loss: 0.0918 - accuracy: 0.9768 - val_loss: 0.5344 - val_accuracy: 0.8016
Epoch 4/10
25000/25000 [==============================] - 9s 349us/sample - loss: 0.0228 - accuracy: 0.9970 - val_loss: 0.6103 - val_accuracy: 0.8046
Epoch 5/10
25000/25000 [==============================] - 10s 403us/sample - loss: 0.0053 - accuracy: 0.9997 - val_loss: 0.6962 - val_accuracy: 0.8044
Epoch 6/10
25000/25000 [==============================] - 11s 434us/sample - loss: 0.0018 - accuracy: 1.0000 - val_loss: 0.7498 - val_accuracy: 0.8071
Epoch 7/10
25000/25000 [=======================

In [11]:
reverse_word_index = dict([(value,key) for (key,value) in wi.items()])
#print(reverse_word_index)


In [12]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)


(10000, 16)


In [13]:
import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, words):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()


In [14]:
# from google.colab import files
# files.download('vecs.tsv')
# files.download('meta.tsv')